# 5D Example 

In this notebook we will go through an example using the **foxi** code features to evaluate the expected utility of a mock scientific survey. This notebook will assume the reader is familiar with all of the concepts and is intended as a practical demonstration of the code. For more details about the utility functions themselves, as well as the inner workings of **foxi**, one can read the paper it was first developed in here: [https://arxiv.org/abs/1803.09491]. 

To begin with, we will import the module `sys` to append our path towards **foxi**. 

In [25]:
import sys
sys.path.append('/Users/Rob/work/foxi_train/foxisource/') # Give your directory to foxisource here.
from foxi import foxi

# These imports aren't stricly necessary to run foxi but they will be useful in our examples.
import numpy as np
from scipy.stats import multivariate_normal

### Setup of the current experiment

Consider a scientific experiment over a 5D parameter space. Fitting a statistical model to the input, let us imagine that the posterior distribution is given by a multivariate Gaussian distribution with non-trivial covariances. Let us quickly generate $10^3$ samples from this 5D posterior. Feel free to change these settings to check the final results.

In [26]:
# Choose a state of low information to start with.
mock_posterior_best_fit = np.asarray([ 1.0, 2.0, 4.0, 0.5, 2.5]) 
mock_posterior_fisher_matrix = np.asarray([[ 2.0, 0.0, 0.0, 0.0, 0.0], 
                                           [ 0.0, 3.0, 0.0, 0.0, 0.0], # This obviously assumes that the  
                                           [ 0.0, 0.0, 5.0, 0.0, 0.0], # covariance matrix is constant 
                                           [ 0.0, 0.0, 0.0, 4.0, 0.0], # with respect to the parameters.     
                                           [ 0.0, 0.0, 0.0, 0.0, 2.0]])  
 
# Quick inversion to generate the samples and mimic some weights too.        
mock_posterior_covariance_matrix = np.linalg.inv(mock_posterior_fisher_matrix)
mock_posterior_samples = np.random.multivariate_normal(mock_posterior_best_fit,mock_posterior_covariance_matrix,10**4)
mock_posterior_sample_weights = multivariate_normal.pdf(mock_posterior_samples,mean=mock_posterior_best_fit,cov=mock_posterior_covariance_matrix)
mock_posterior_samples_output = np.insert(mock_posterior_samples,0,mock_posterior_sample_weights,axis=1)

# Let's output this data to a file in the '/foxichains' directory which mimics a real MCMC output.
np.savetxt('/Users/Rob/work/foxi_train/foxichains/mock_posterior_samples.txt', mock_posterior_samples_output, delimiter='\t')

### Setup of the model priors

We should now generate a series of toy model priors which can span the parameter space. These should represent a true collection of theoretical models but, in this case for simplicity and to capture the essential effects of the whole model space, we will choose `Nm` models that all have uniform hypersphere priors all with radius `Rm`, and positions drawn from a Gaussian hyperprior centred on `mock_posterior_best_fit`. Let us now quickly generate $100$ samples for each of these models and save them to files so that **foxi** can read them in. 


In [27]:
# Feel free to vary these (though consider the number of model pairs to compare grows like Nm*(Nm-1)/2).
Nm = 10  
Rm = 0.1
    
hyperprior_covariance_matrix = np.asarray([[ 0.05, 0.01, 0.01, 0.00, 0.00], 
                                           [ 0.01, 0.05, 0.01, 0.00, 0.00], # Different values here correspond to   
                                           [ 0.01, 0.01, 0.05, 0.00, 0.00], # different model space alignments. 
                                           [ 0.00, 0.00, 0.00, 0.05, 0.00],      
                                           [ 0.00, 0.00, 0.00, 0.00, 0.05]])  

# Generate the positions.
mock_prior_positions = np.random.multivariate_normal(mock_posterior_best_fit,hyperprior_covariance_matrix,Nm)

# Generate the 5D hypersphere samples and output to text files in the '/foxipriors' directory.
for im in range(0,Nm):
    angle1 = np.random.uniform(0,np.pi,size=10**2)
    angle2 = np.random.uniform(0,np.pi,size=10**2)
    angle3 = np.random.uniform(0,np.pi,size=10**2)
    angle4 = np.random.uniform(0,2.0*np.pi,size=10**2)
    parameter1 = Rm*np.cos(angle1) + mock_prior_positions[im][0]
    parameter2 = Rm*np.sin(angle1)*np.cos(angle2) + mock_prior_positions[im][1]
    parameter3 = Rm*np.sin(angle1)*np.sin(angle2)*np.cos(angle3) + mock_prior_positions[im][2]
    parameter4 = Rm*np.sin(angle1)*np.sin(angle2)*np.sin(angle3)*np.cos(angle4) + mock_prior_positions[im][3]
    parameter5 = Rm*np.sin(angle1)*np.sin(angle2)*np.sin(angle3)*np.sin(angle4) + mock_prior_positions[im][4]
    mock_prior_samples = np.asarray([parameter1,parameter2,parameter3,parameter4,parameter5]).T
    np.savetxt('/Users/Rob/work/foxi_train/foxipriors/mock_prior' + str(im+1) + '_samples.txt', mock_prior_samples, delimiter='\t')    

### Setup of the forecast Fisher matrix

Let us assume that the future likelihood will be a multivariate Gaussian over the parameters as well. The code can of course accomodate any specified fiduical point-dependent matrix, but it will be more instructive to simplify things in this way. In this instance, let us also model how the Fisher matrix varies with respect to the fiducial points with a polynomial. 

In [38]:
# Specify the polynomial baheviour of the forecast Fisher matrix with respect to the fiducial points.
def fisher_matrix(fiducial_point):
    return np.asarray([[2.0+abs(fiducial_point[0])**2.0, 0.0,0.0,0.0,0.0],
                       [0.0,10.0+abs(fiducial_point[1])**2.0,0.0,0.0,0.0],
                       [0.0,0.0,5.0+abs(fiducial_point[2])**2.0, 0.0,0.0],
                       [0.0,0.0,0.0,10.0+abs(fiducial_point[3])**2.0,0.0],
                       [0.0,0.0,0.0,0.0,10.0+abs(fiducial_point[4])**2.0]]) 

### Running the main foxi algorithm

We are now ready to run **foxi** with our samples and forecast Fisher matrix. The first thing to do is to run a new instance of the `foxi` class like so

In [29]:
foxi_instance = foxi('/Users/Rob/work/foxi_train') # Give your directory to foxi here.

The next step is to specify where our posterior samples of the current data are, how many samples to read in, what weights they carry and identify if there are any transformations to be done on each column (the latter will be no in this case). 

In [30]:
chains_filename = 'mock_posterior_samples.txt'

# Note that the column numbers start from 0...
parameter_column_numbers = [1,2,3,4,5]
weights_column_number = 0

# Simply set this to the number of samples generated - this can be useful to get results out as a sanity check.
number_of_samples_to_read_in = 10**3

foxi_instance.set_chains(chains_filename,
                         parameter_column_numbers,
                         number_of_samples_to_read_in,
                         weights_column=weights_column_number, # All points are given weight 1 if this is ignored.
                         column_types=None) # No transformations needed here. 
                                            # One could have ['flat','log10','log'] specified for each column.

Now that the posterior chains have been set, we can do the same for our list of prior samples.

In [31]:
# List the model file names to compute the expected utilities for.
model_name_list = ['mock_prior' + str(im+1) + '_samples.txt' for im in range(0,Nm)]

# List the column numbers to use for each file of prior samples.
prior_column_numbers = [[0,1,2,3,4] for im in range(0,Nm)]

# Once again, simply set this to the number of samples we made earlier for each prior.
number_of_prior_points_to_read_in = 10**2

foxi_instance.set_model_name_list(model_name_list,
                                  prior_column_numbers,
                                  number_of_prior_points_to_read_in,
                                  prior_column_types=None) # Once again, no transformations needed here. 

Using the statsmodels module: http://www.statsmodels.org/stable/index.html
The Kernel Density Bandwidth for each model listed in each dimension:

Model 0: [0.04475697 0.03007876 0.02293838 0.01620533 0.01824696]
Model 1: [0.04402931 0.03190598 0.02199352 0.01705286 0.01398627]
Model 2: [0.04403217 0.03385921 0.02196357 0.01509964 0.01411397]
Model 3: [0.04639548 0.03050075 0.02088313 0.01642939 0.01509258]
Model 4: [0.04587787 0.0297153  0.02181327 0.0148354  0.01824378]
Model 5: [0.04611706 0.03271811 0.01940892 0.01535048 0.01501017]
Model 6: [0.04545002 0.03001187 0.02399223 0.01349952 0.01562673]
Model 7: [0.04438425 0.03047156 0.02151398 0.01894682 0.01628939]
Model 8: [0.04737934 0.03013409 0.02117865 0.01367327 0.01519263]
Model 9: [0.0438121  0.03384669 0.01934215 0.01911765 0.01450829]


Notice that the output here is from a quick KDE of each set of prior samples. How these are used depends on the specific numerical situation, which is once again covered in more detail in [https://arxiv.org/abs/1803.09491]. Our final step before running **foxi** is to give it a python function which returns the forecast Fisher matrix at each fiducial point (i.e. evaluated at each point of the current data chains). We do this like so

In [32]:
foxi_instance.set_fisher_matrix(fisher_matrix)

All of the necessary settings have now been applied for a full run of the **foxi** code. Depending on how many points were introduced at the level of the priors and chains, as well as the length of time it takes to evaluate the forecast Fisher matrix at each fiducial point, we may wish to continue on a compute cluster. For this simple example we have chosen, it should be enough to simply run locally on a laptop in less than 10 minutes. Our main run command is the following

In [33]:
mix_models = True # Set this to 'True' so that the expected utilities for all possible model pairs are calculated.
                  # The default is 'False' which calculates the utilities all with respect to the reference model
                  # here the 0-element in 'model_name_list'.
        
foxi_instance.run_foxifish(mix_models=mix_models)

                                          
>>>>>>>>>>>                               
>>>       >>                              
>>                                        
>>                                  >>    
>>>>>>>>>    >>>>>>>    >>>    >>>        
>>          >>     >>     >>  >>    >>    
>>         >>       >>     >>>>     >>    
>>         >>       >>    >>>>      >>    
>>          >>     >>    >>  >>     >>    
>>           >>>>>>>   >>>    >>>   >>>   
                                          
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
       Author: Robert J. Hardwick         
      DISTRIBUTED UNDER MIT LICENSE       
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
                                          
    NOW WORKING WITH FISHER FORECASTS     
                                          
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

Number of maxed-out evidences for each model:

Model 0: 0
Model 1: 0
Model 2: 0
Model 3: 0
Model 4: 0
Model 5: 0
Model 6: 0
Model 7: 0
Mode

Don't be too alarmed by a few error messages about dividing by zero: these occur infrequently due to excessively low densities appearing in the numerical procedure but are automatically discarded as points from the main result. The algorithm should terminate with a `Number of maxed-out evidences for each model:` message. This message refers to the number of times when the model was found to be ruled out at the level of its maximum likelihood. 

### Post-processing and other foxi features

This last step was the longest running, from now onwards the post-processing of the results from **foxi** will always be quick enough to run locally on a laptop. The next step is to use `rerun_foxi` to analyse the output that `run_foxifish` dumped into the `/foxioutput` directory.

In [34]:
foxiplot_file_name = 'foxiplots_data_mix_models.txt' # This is the generic name that 'run_foxifish' will set, change
                                                     # this to whatever you like as long as the file is in '/foxioutput'.
                                                     # If 'mix_models = False' then remove the 'mix_models' tag.

# Set this to the number of samples generated - useful to vary to check convergence though we will make plots later.
number_of_foxiplot_samples_to_read_in = 10**3 

# We must set this feature to 'flat' in each column to perform no post-processing transformation that reweights the chains.
# This can be a little redundant as it makes more numerical sense to simply generate new chains. 
post_chains_column_types = ['flat','flat','flat','flat','flat']

# Set this to 'True' for the output to include fully-formatted LaTeX tables!
TeX_output = True

# For the truly lazy - you can set the TeX name for each model in the table output too.
model_TeX_names = [r'${\cal M}_' + str(i) + r'$' for i in range(0,Nm)]

foxi_instance.rerun_foxi(foxiplot_file_name,number_of_foxiplot_samples_to_read_in,post_chains_column_types,model_name_TeX_input=model_TeX_names,TeX_output=TeX_output)

                                          
>>>>>>>>>>>                               
>>>       >>                              
>>                                        
>>                                  >>    
>>>>>>>>>    >>>>>>>    >>>    >>>        
>>          >>     >>     >>  >>    >>    
>>         >>       >>     >>>>     >>    
>>         >>       >>    >>>>      >>    
>>          >>     >>    >>  >>     >>    
>>           >>>>>>>   >>>    >>>   >>>   
                                          
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
       Author: Robert J. Hardwick         
      DISTRIBUTED UNDER MIT LICENSE       
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
                                          
Just read in a `mix_models' foxiplot data file with data dimension 5 and the number of model pairs is 55


Once again, don't be too alarmed by a few value error messages. If there are any error messages related to converting strings to floats, this is likely to do with a delimiter problem in the text files that were read in. Tab delimited data should work fine (or at least give a number of spaces). Note also that the number of model pairs quoted here is actually the number of unique pairs + the auto-pairs (i.e. Model_i - Model_j as well as Model_i - Model_i) so this number is an additional `+N_m` larger than the number of unique pairs. A summary of the main results can be read from this file

In [35]:
print(open('/Users/Rob/work/foxi_train/foxioutput/foxiplots_data_summary.txt', 'r').read())

 [<|lnB|>_1, <|lnB|>_2, ...] = [0.         7.13949588 3.54252784 1.96244785 3.11499612 4.48519101
 3.38517049 5.45135066 4.47040482 4.80322344 0.         6.89323786
 6.42265575 5.74799757 4.08894828 5.88328596 6.19814586 4.9617276
 4.31835716 0.         2.06543351 4.85277293 4.38120967 3.81736211
 5.86051277 4.98013573 3.99211372 0.         3.39313949 3.97178291
 3.24042666 5.45303046 4.12438734 3.91150725 0.         5.26961806
 5.18446365 7.36248621 5.76247833 5.66165391 0.         2.29951833
 3.48347557 2.9438242  1.31189337 0.         3.5882871  3.42725462
 2.43612895 0.         2.4016791  3.74371312 0.         3.01221195
 0.        ]
 [DECI_1, DECI_2, ...] = [0.    0.551 0.247 0.054 0.169 0.34  0.225 0.426 0.346 0.374 0.    0.52
 0.505 0.459 0.309 0.477 0.498 0.398 0.341 0.    0.078 0.394 0.311 0.258
 0.458 0.384 0.297 0.    0.227 0.284 0.21  0.435 0.295 0.289 0.    0.431
 0.441 0.566 0.467 0.452 0.    0.097 0.233 0.172 0.005 0.    0.248 0.245
 0.102 0.    0.105 0.277 0.    0.18  0

One thing to avoid in these results is if the quoted expected Kullback-Leibler divergence `<DKL>` is of roughly the same value as the natural log of the number of points in the posterior chains - which in this example is $\ln (10^3) \simeq 6.9$. More chain points must be used for larger values since otherwise the value of `<DKL>` is only a lower bound. 

A nice feature, if one sets `TeX_output = True` in `rerun_foxi` above, is the output of a fully-formatted $\LaTeX$ table incorporating of all of these results. This can be found in the following file

In [36]:
print(open('/Users/Rob/work/foxi_train/foxioutput/foxiplots_data_summary_TeX.txt', 'r').read())

\begin{table}
\centering
\begin{tabular}{|c|c|c|c|c|c|}
\cline{2-6}
\multicolumn{1}{c}{\cellcolor{red!55}} & \multicolumn{5}{|c|}{Data Name}  \\      \hline
\backslashbox{${\cal M}_\beta$ - ${\cal M}_\gamma$}{$\langle  U\rangle$}  & $\langle \vert \ln {\rm B}_{\beta \gamma}\vert \rangle$ & $\langle \vert \ln {\rm B}_{\beta \gamma} \vert \rangle_{{}_{\rm ML}}$ & $\mathscr{D}_{\beta \gamma}$ & $\mathscr{D}_{\beta \gamma}\vert_{{}_{\rm ML}}$ & $r_{{}_\mathrm{ML}}$ \\     \hline\hline
${\cal M}_0$ - ${\cal M}_0$ & $0.0^{+0.0}_{-0.0}$ & $0.0^{+0.0}_{-0.0}$ & 0.0 & 0.0 & 0.79  \\      \hline
${\cal M}_1$ - ${\cal M}_0$ & 7.14 $\pm$ 6.0 & 5.57 $\pm$ 3.68 & 0.55 & 0.49 & 0.63  \\      \hline
${\cal M}_2$ - ${\cal M}_0$ & 3.54 $\pm$ 3.08 & 2.53 $\pm$ 1.78 & 0.25 & 0.12 & 0.72  \\      \hline
${\cal M}_3$ - ${\cal M}_0$ & 1.96 $\pm$ 1.63 & 1.25 $\pm$ 0.85 & 0.05 & 0.0 & 0.72  \\      \hline
${\cal M}_4$ - ${\cal M}_0$ & 3.11 $\pm$ 2.84 & 1.84 $\pm$ 1.19 & 0.17 & 0.01 & 0.73  \\      \hline
${\ca

Another useful tool provided by **foxi** is to generate plots illustrating the numerical convergence (or lack thereof) of the calculated expected utilities.

In [57]:
# Simply specify the number of bins in which to re-calculate the expected utilty with more samples. 
number_of_bins = 50

# We have already specified the other inputs here so let's simply generate the plots!
foxi_instance.plot_convergence(foxiplot_file_name,number_of_bins,number_of_foxiplot_samples_to_read_in,post_chains_column_types)

The output can be found in `/foxiplots/foxiplot_convergence.pdf`.

### Computing analytic expected utility estimates with foxi

In the regime where the number of samples from the current posterior is large enough (i.e. such that the expected Kullback-Leibler divergence is less than $\ln (\#_{\rm samples})$) and the prior volume of each model is small enough to be effectively described as a single point in parameter space (in the present example this is modelled with `Rm` being small), the expected utilities `<DKL>` and `<|lnB|>` (with their standard deviations) can be compared with analytic estimates of the same quantities. These formulae rely on knowledge of the Fisher estimate for the current posterior and are built into **foxi**. One can obtain them through using the following command

In [37]:
stepsizes = [np.sqrt(mock_posterior_covariance_matrix[0,0]),
             np.sqrt(mock_posterior_covariance_matrix[1,1]), # Set the stepsizes in calculating the 
             np.sqrt(mock_posterior_covariance_matrix[2,2]), # variation of the forecast Fisher matrix
             np.sqrt(mock_posterior_covariance_matrix[3,3]), # with respect to the fiducial points.
             np.sqrt(mock_posterior_covariance_matrix[4,4])]

# The rest of the inputs have been determined in our example previously
foxi_instance.analytic_estimates(mock_prior_positions,mock_posterior_best_fit,mock_posterior_fisher_matrix,stepsizes)

<DKL> estimate: 2.762575144483158
Sum_i <|lnB|>_i estimate: 107.21283488738516
<|lnB|>_0 estimate: 3.0846895430056622
<|lnB|>_1 estimate: 1.6630932970048868
<|lnB|>_2 estimate: 1.2697094930439952
<|lnB|>_3 estimate: 1.4314272313113006
<|lnB|>_4 estimate: 2.711081199457574
<|lnB|>_5 estimate: 2.231660640226594
<|lnB|>_6 estimate: 2.166398840619568
<|lnB|>_7 estimate: 2.0501646182900775
<|lnB|>_8 estimate: 2.528960770375072
<|lnB|>_9 estimate: 3.260917809550255
<|lnB|>_10 estimate: 3.3873423886539005
<|lnB|>_11 estimate: 2.0088919703992003
<|lnB|>_12 estimate: 3.301920472382519
<|lnB|>_13 estimate: 3.5341405462830005
<|lnB|>_14 estimate: 2.03660482752888
<|lnB|>_15 estimate: 2.3510589134622797
<|lnB|>_16 estimate: 3.189378705439924
<|lnB|>_17 estimate: 1.5290931484273684
<|lnB|>_18 estimate: 2.756999412755398
<|lnB|>_19 estimate: 2.9161082312475686
<|lnB|>_20 estimate: 2.986223996679991
<|lnB|>_21 estimate: 2.7437074203045064
<|lnB|>_22 estimate: 2.6646327021513505
<|lnB|>_23 estimate: 2

The agreement between the analytic estimates and the output from **foxi** can typically be improved by reducing `Rm` and increasing the number of points drawn from the current posterior, though the estimates have limited accuracy themeselves depending on the number of derivatives that can be taken in the forecast Fisher matrix. The comparison between these separate results can be a useful guide when obtaining forecasts from **foxi** to the required accuracy.